In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.common.by import  By 
from selenium.webdriver.support import  expected_conditions as EC 

import pandas as pd
import numpy as np
import time
import datetime

In [11]:
#建立数据库
columns=['时间','发布日期','作者名','作者类型','粉丝数','投稿数','视频名','分区','子分区','视频类型','自制or转载','标签','视频长度','播放量','点赞数','投币数','收藏数','转发数','评论数','弹幕数','有无分集']
channel_yinyue_yuanchuang=pd.DataFrame(columns=columns)
channel_yinyue_yuanchuang

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集


In [3]:
driver=webdriver.Chrome()
driver.get('https://passport.bilibili.com/login') 

In [4]:
#登录
driver.find_element_by_xpath('//*[@id="login-username"]').send_keys('15717506191')
driver.find_element_by_xpath('//*[@id="login-passwd"]').send_keys('xy000000')
driver.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
#需要手动过验证码

In [5]:
driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[3]/div/a/span').click()  #选择“音乐”频道

In [6]:
#获得分区名
channel=driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[3]/div/a/span').text
channel=channel.strip('0123456789+')
channel

'音乐'

In [7]:
driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[2]/a').click()  #选择“特摄”子频道
sub_channel=driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[2]/a').text
sub_channel

'原创音乐'

In [8]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[1]/ul/a[2]/li').click() #让视频按热度排序

In [12]:
print(datetime.datetime.now())

for month in range(1,11): 
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div/span').click()
    period=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).text
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]/div/div[2]/div[3]/a'))) 
    for page in range(5):    #爬取每个频道1月到10月每个月播放量前100的视频

        video_loc=1
        while video_loc<21:

            ### 判断作者类型###
            
            
            driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).click()
            driver.switch_to_window(driver.window_handles[-1])
            
            
            author_web_show='No'
            try:
                i=0
                while author_web_show=='No':
                    driver.refresh()
                    i+=1
                    try:
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="n-fs"]')))  #看作者名是否出现
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[3]')))  #看投稿数是否出现
                        author_web_show='Yes'
                    except:
                        author_web_show='No' 
                        if i>3:  
                            raise Exception  #刷新几次后仍爬不了说明用户已注销

                try:
                    driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m organization-auth"]')
                    author_type='机构认证'

                except:
                    try:
                        driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m personal-auth"]')
                        author_type='个人认证'
                    except:
                        author_type='无'

                fans_num=driver.find_element_by_xpath('//*[@id="n-fs"]').text  #获取作者粉丝数

                ### 获取作者投稿数 ###
                driver.find_element_by_xpath('//*[@class="n-num"]').click()
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span')))  
                video_num=driver.find_element_by_xpath('//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span').text

                driver.close()
                driver.switch_to_window(driver.window_handles[-1])


                view_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[1]/span'%video_loc).text #获取播放量
                real_dan_mu_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[2]/span'%video_loc).text #获取弹幕数
                author=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).text #获取作者名
                driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/a'%video_loc).click() #进入视频界面  
                driver.switch_to_window(driver.window_handles[-1]) 

                try:
                    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))  #若出现视频被删除的情况则退出

                    video_web_show='No'
                    while video_web_show=='No':
                        driver.refresh()

                        try:
                            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@class="bilibili-player-video-time-total"]'))) #以视频长度出现时间判定网页是否加载完
                            video_web_show='Yes'

                            try:      
                                driver.find_element_by_xpath('//*[@id="multi_page"]')   #测试是否有分集视频
                                multi='是'

                            except:
                                multi='否'

                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            driver.execute_script('window.scrollTo(document.body.scrollHeight,0)') #再调回最上面是因为，视频长度的元素必须在屏幕内才可以被提取，否则为空

                            bv=driver.current_url[33:44] # 获取BV号（BV号在网址中）
                            print('开始BV号为%s视频属性的抓取'%bv)
                            video_name=driver.find_element_by_xpath('//*[@id="viewbox_report"]/h1/span').text #获取视频名
                            
                            ### 获取发布日期 ###
                            release_date=driver.find_element_by_xpath('//*[@id="viewbox_report"]/div/span[3]').text
                            
                            ### 获取视频类型 ###
                            try:
                                video_type=driver.find_element_by_xpath('//*[@class="activity"]').text
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type=video_type+'和合作视频'
                                except:
                                    pass
                            except:
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type='合作视频'
                                except:
                                    video_type='普通视频'


                            ### 获取点赞数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text=='点赞':
                                like_num=0
                            else:
                                like_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text 

                            ### 获取投币数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text=='投币':
                                coin_num=0
                            else:
                                coin_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text

                            ### 获取收藏数 ###             
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text=='收藏':
                                collect_num=0
                            else:
                                collect_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text

                            ### 获取转发数 ###       
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text=='分享':
                                forward_num=0
                            else:
                                forward_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text

                            ### 获取视频长度 ###

                            video_length=driver.find_element_by_xpath('//*[@class="bilibili-player-video-time-total"]').text  

                            ### 判断自制还是转载 ###
                            driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').click()
                            try:
                                driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/div[2]/div[2]').text
                                self_made='自制'
                            except:
                                self_made='转载'
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/i').click()

                            ### 获取标签 ###
                            k=1
                            labels=np.array
                            while True:
                                try:
                                    labels=np.append(labels,driver.find_element_by_xpath('//*[@id="v_tag"]/ul/li[%d]'%k).text)
                                    k+=1
                                except: 
                                    break
                            labels=labels[1:]

                            ### 获取评论数 ###
                            driver.execute_script('window.scrollTo(0, 700)')        
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[2]'))) #保证"评论"二字能加载出来                                                   
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div')))
                            if driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/div[4]/div').text=='没有更多信息':    #应对评论区被关闭的情况
                                comment_num='NaN'
                            else:
                                while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                    try:
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div[1]')))
                                        while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                            time.sleep(0.1) #很奇怪，评论数可以定位的一瞬间并不能爬取，会返回空
                                        comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text     
                                    except:
                                        driver.refresh()    #有的时候“评论”二字出现，评论数不出现，页面好像卡住了
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))
                                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[1]')))

                                comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text

                            channel_yinyue_yuanchuang.loc[bv]=[period,release_date,author,author_type,fans_num,video_num,video_name,channel,sub_channel,video_type,self_made,labels,video_length,view_num,like_num,coin_num,collect_num,forward_num,comment_num,real_dan_mu_num,multi]

                            print('BV号为%s的视频属性抓取结束'%bv)

                            driver.close()
                            driver.switch_to_window(driver.window_handles[-1])

                        except:
                            video_web_show='No' 
                except:
                    driver.close()
                    driver.switch_to_window(driver.window_handles[-1])
            except:
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
            
            video_loc+=1


        driver.find_element_by_xpath('//*[@class="nav-btn iconfont icon-arrowdown3"]').click() #点击下一页
        WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]')))  #点击下一页后等页面刷新
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[2]').click()  #回到第1页
    WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]')))
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]').click()  #发现很多月份漏爬了约20个，而且恰好是第一页，所以再点一次第一页
        
        
print(datetime.datetime.now()) 

2020-12-20 08:54:44.958226


<ipython-input-12-b6965743a003>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-12-b6965743a003>:54: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-12-b6965743a003>:61: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1p7411Y7BC视频属性的抓取
BV号为1p7411Y7BC的视频属性抓取结束


<ipython-input-12-b6965743a003>:181: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1KJ411C7Q6视频属性的抓取
BV号为1KJ411C7Q6的视频属性抓取结束
开始BV号为1aJ411V7hd视频属性的抓取
BV号为1aJ411V7hd的视频属性抓取结束
开始BV号为1KJ411C7qF视频属性的抓取
BV号为1KJ411C7qF的视频属性抓取结束
开始BV号为117411r7R1视频属性的抓取
BV号为117411r7R1的视频属性抓取结束
开始BV号为1P7411B71g视频属性的抓取
BV号为1P7411B71g的视频属性抓取结束
开始BV号为1XJ41157y8视频属性的抓取
BV号为1XJ41157y8的视频属性抓取结束
开始BV号为1AJ411L7Bh视频属性的抓取
BV号为1AJ411L7Bh的视频属性抓取结束
开始BV号为1zJ411E77i视频属性的抓取
BV号为1zJ411E77i的视频属性抓取结束
开始BV号为1j7411e7uG视频属性的抓取
BV号为1j7411e7uG的视频属性抓取结束
开始BV号为1uJ411V74G视频属性的抓取
BV号为1uJ411V74G的视频属性抓取结束
开始BV号为1KJ411C7Yh视频属性的抓取
BV号为1KJ411C7Yh的视频属性抓取结束
开始BV号为137411z7Js视频属性的抓取
BV号为137411z7Js的视频属性抓取结束
开始BV号为1LJ411p7MD视频属性的抓取
BV号为1LJ411p7MD的视频属性抓取结束
开始BV号为1dJ411j7wN视频属性的抓取
BV号为1dJ411j7wN的视频属性抓取结束
开始BV号为19J41157hM视频属性的抓取
BV号为19J41157hM的视频属性抓取结束
开始BV号为1XJ411h7EY视频属性的抓取
BV号为1XJ411h7EY的视频属性抓取结束
开始BV号为1HJ411H7Rj视频属性的抓取
BV号为1HJ411H7Rj的视频属性抓取结束
开始BV号为1KJ411j78b视频属性的抓取
BV号为1KJ411j78b的视频属性抓取结束
开始BV号为17J411h7pU视频属性的抓取
BV号为17J411h7pU的视频属性抓取结束
开始BV号为1y7411i7MG视频属性的抓取
开始BV号为1y7411i7MG视频属性的抓取
开始BV号为1y7411i7MG视频属性的抓取
BV号为1y7411i7MG的视

...

<ipython-input-12-b6965743a003>:190: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1j54y1X7Gd视频属性的抓取
BV号为1j54y1X7Gd的视频属性抓取结束
开始BV号为1dA41147fF视频属性的抓取
BV号为1dA41147fF的视频属性抓取结束
开始BV号为1jT4y1u7bk视频属性的抓取
BV号为1jT4y1u7bk的视频属性抓取结束
开始BV号为1Ez411q7da视频属性的抓取
BV号为1Ez411q7da的视频属性抓取结束
开始BV号为13A411t7Fb视频属性的抓取
BV号为13A411t7Fb的视频属性抓取结束
开始BV号为1hk4y1k7uh视频属性的抓取
BV号为1hk4y1k7uh的视频属性抓取结束
开始BV号为1Te411W7s1视频属性的抓取
BV号为1Te411W7s1的视频属性抓取结束
开始BV号为14K4y1b7qo视频属性的抓取
BV号为14K4y1b7qo的视频属性抓取结束
开始BV号为1YQ4y1N7qj视频属性的抓取
BV号为1YQ4y1N7qj的视频属性抓取结束
开始BV号为1F5411x7v8视频属性的抓取
BV号为1F5411x7v8的视频属性抓取结束
开始BV号为1Ma4y1v7RF视频属性的抓取
BV号为1Ma4y1v7RF的视频属性抓取结束
开始BV号为1TK4y1476o视频属性的抓取
BV号为1TK4y1476o的视频属性抓取结束
开始BV号为1R5411W7BK视频属性的抓取
BV号为1R5411W7BK的视频属性抓取结束
开始BV号为1FZ4y1H79z视频属性的抓取
BV号为1FZ4y1H79z的视频属性抓取结束
开始BV号为1Wv411672y视频属性的抓取
BV号为1Wv411672y的视频属性抓取结束
开始BV号为1La4y1Y7H7视频属性的抓取
BV号为1La4y1Y7H7的视频属性抓取结束
开始BV号为1pK411s7s4视频属性的抓取
BV号为1pK411s7s4的视频属性抓取结束
开始BV号为1az4y1Q7aB视频属性的抓取
BV号为1az4y1Q7aB的视频属性抓取结束
开始BV号为13A411i7Xi视频属性的抓取
BV号为13A411i7Xi的视频属性抓取结束
开始BV号为1dC4y1a7ZG视频属性的抓取
BV号为1dC4y1a7ZG的视频属性抓取结束
开始BV号为1kT4y1J7mB视频属性的抓取
BV号为1kT4y1J7mB的视

BV号为12i4y1G7X5的视频属性抓取结束
开始BV号为1rV41167VQ视频属性的抓取
BV号为1rV41167VQ的视频属性抓取结束
开始BV号为1AK411H7Bo视频属性的抓取
BV号为1AK411H7Bo的视频属性抓取结束
开始BV号为1gf4y1R7uV视频属性的抓取
BV号为1gf4y1R7uV的视频属性抓取结束
开始BV号为1d54y1v7Cc视频属性的抓取
BV号为1d54y1v7Cc的视频属性抓取结束
开始BV号为1Yt4y197Pg视频属性的抓取
BV号为1Yt4y197Pg的视频属性抓取结束
开始BV号为1Fg4y1i7To视频属性的抓取
BV号为1Fg4y1i7To的视频属性抓取结束
开始BV号为1yV411B7UF视频属性的抓取
BV号为1yV411B7UF的视频属性抓取结束
开始BV号为1UC4y187yr视频属性的抓取
BV号为1UC4y187yr的视频属性抓取结束
开始BV号为1ot4y197BJ视频属性的抓取
BV号为1ot4y197BJ的视频属性抓取结束
开始BV号为1D54y1q7So视频属性的抓取
BV号为1D54y1q7So的视频属性抓取结束
开始BV号为1iK4y1s77c视频属性的抓取
BV号为1iK4y1s77c的视频属性抓取结束
开始BV号为1zv411i7Ek视频属性的抓取
BV号为1zv411i7Ek的视频属性抓取结束
开始BV号为1hp4y1i7WW视频属性的抓取
BV号为1hp4y1i7WW的视频属性抓取结束
开始BV号为1654y1v73f视频属性的抓取
BV号为1654y1v73f的视频属性抓取结束
开始BV号为1mz4y1D7Tp视频属性的抓取
BV号为1mz4y1D7Tp的视频属性抓取结束
开始BV号为1tZ4y1T7eD视频属性的抓取
BV号为1tZ4y1T7eD的视频属性抓取结束
开始BV号为1t54y1q7g9视频属性的抓取
BV号为1t54y1q7g9的视频属性抓取结束
开始BV号为1xK4y1x7H9视频属性的抓取
BV号为1xK4y1x7H9的视频属性抓取结束
开始BV号为1ct4y1X7uT视频属性的抓取
BV号为1ct4y1X7uT的视频属性抓取结束
开始BV号为11v411q7Jq视频属性的抓取
BV号为11v411q7Jq的视频属性抓取结束
开始BV号为1JT4y1j7SU

BV号为1qV41117Zw的视频属性抓取结束
开始BV号为1Wh411X7Ka视频属性的抓取
BV号为1Wh411X7Ka的视频属性抓取结束
开始BV号为1464y1F7H6视频属性的抓取
BV号为1464y1F7H6的视频属性抓取结束
开始BV号为19V411m79M视频属性的抓取
BV号为19V411m79M的视频属性抓取结束
开始BV号为1rk4y1y76X视频属性的抓取
BV号为1rk4y1y76X的视频属性抓取结束
开始BV号为1yp4y1a7RX视频属性的抓取
BV号为1yp4y1a7RX的视频属性抓取结束
开始BV号为1Zi4y1776h视频属性的抓取
BV号为1Zi4y1776h的视频属性抓取结束
开始BV号为1HK411P7Er视频属性的抓取
BV号为1HK411P7Er的视频属性抓取结束
开始BV号为1D54y1y79L视频属性的抓取
BV号为1D54y1y79L的视频属性抓取结束
开始BV号为1aT4y1A7o9视频属性的抓取
BV号为1aT4y1A7o9的视频属性抓取结束
开始BV号为1EA41177aK视频属性的抓取
BV号为1EA41177aK的视频属性抓取结束
开始BV号为1Vk4y117Dz视频属性的抓取
BV号为1Vk4y117Dz的视频属性抓取结束
开始BV号为1R5411b791视频属性的抓取
BV号为1R5411b791的视频属性抓取结束
开始BV号为18p4y1e7Lh视频属性的抓取
BV号为18p4y1e7Lh的视频属性抓取结束
开始BV号为1Fp4y1e76X视频属性的抓取
开始BV号为1Fp4y1e76X视频属性的抓取
开始BV号为1Fp4y1e76X视频属性的抓取
开始BV号为1Fp4y1e76X视频属性的抓取
BV号为1Fp4y1e76X的视频属性抓取结束
开始BV号为1i54y127aL视频属性的抓取
BV号为1i54y127aL的视频属性抓取结束
开始BV号为1Ct4y1i7Ja视频属性的抓取
BV号为1Ct4y1i7Ja的视频属性抓取结束
开始BV号为1Ma4y1E73b视频属性的抓取
BV号为1Ma4y1E73b的视频属性抓取结束
开始BV号为1bv411172T视频属性的抓取
开始BV号为1bv411172T视频属性的抓取
BV号为1bv411172T的视频属性抓取结束
开始BV号为18K4y1a7B5

BV号为1UK411A728的视频属性抓取结束
开始BV号为1XD4y1R74u视频属性的抓取
BV号为1XD4y1R74u的视频属性抓取结束
开始BV号为1YK411N7aA视频属性的抓取
BV号为1YK411N7aA的视频属性抓取结束
开始BV号为1Ay4y187r8视频属性的抓取
BV号为1Ay4y187r8的视频属性抓取结束
开始BV号为1Rz4y1Z7Ph视频属性的抓取
BV号为1Rz4y1Z7Ph的视频属性抓取结束
开始BV号为1fa4y1W7QP视频属性的抓取
BV号为1fa4y1W7QP的视频属性抓取结束
2020-12-20 19:12:34.003916


In [10]:
driver.switch_to_window(driver.window_handles[-1])

<ipython-input-10-dd0a1399d54d>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


In [13]:
channel_yinyue_yuanchuang.groupby('时间').agg('count')

,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
时间,,,,,,,,,,,,,,,,,,,,
10月,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97
1月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
2月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
3月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
4月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
5月,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97
6月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
7月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
8月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100


In [14]:
channel_yinyue_yuanchuang.to_json('channel_yinyue_yuanchuang.json')

In [2]:
channel_yinyue_yuanchuang=pd.read_json('channel_yinyue_yuanchuang.json')

In [5]:
pd.set_option('display.max_rows', None)

In [6]:
channel_yinyue_yuanchuang[channel_yinyue_yuanchuang['时间']=='8月']

,时间,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
1k54y1U79J,8月,药水哥y,个人认证,64.8万,44,【药水歌】我出门总是带着五瓶药水~,音乐,原创音乐,合作视频,自制,"[动感视频, 原创歌曲, RAP, 说唱, 音乐, 原创音乐, 音乐, 正能量, 药水哥, ...",03:06,494.5万,29.3万,15.6万,8.5万,5.7万,11291,2.3万,否
1qV411U7sm,8月,老坛胡说,个人认证,228.6万,197,谁定义,音乐,原创音乐,合作视频,自制,"[原创歌曲, 说唱, 搞笑, 音乐, 原创音乐, 尤超白, 老坛胡说]",05:14,385.9万,37.1万,29.9万,16.5万,2.4万,8490,2.5万,是
1SK411T7KB,8月,哔哩哔哩娘,个人认证,52.9万,85,【2233生日】时光软糖,音乐,原创音乐,合作视频,自制,"[原创歌曲, 音乐, 原创音乐, 哔哩哔哩娘, 2233, 生日, 十周年, bilibil...",04:25,268.3万,39.1万,26.7万,16.1万,2.1万,19877,13.0万,否
1Cv411v7Po,8月,黄龄,个人认证,169.8万,56,【黄龄/ilem】《达拉崩吧》喵千岁版来啦！一人分饰多角挑战4种音色,音乐,原创音乐,合作视频,自制,"[黄龄, 乘风破浪的姐姐, 梦幻西游, 原创歌曲, 音乐, 原创音乐, 达拉崩吧, 梦幻西游...",04:02,238.0万,18.7万,10.4万,3.1万,7485,5682,9268,否
1gD4y1m7f6,8月,妙影轻音阁,个人认证,16.7万,127,【188家主团】最后是我（水千丞原著/188男团系列家主团同人出道曲/妙影轻音阁出品）,音乐,原创音乐,合作视频,自制,"[原创歌曲, 音乐, 原创音乐, 188家主团, 水千丞, 188男团, 妙影轻音阁, 最后是我]",04:45,214.0万,17.3万,16.5万,15.2万,4.1万,19512,14.6万,否
1iK4y1e7uL,8月,明日方舟,机构认证,323.7万,80,《明日方舟》EP - ALIVE,音乐,原创音乐,普通视频,自制,"[原创歌曲, 明日方舟, 音乐, 原创音乐, 音乐, 纯音乐]",04:18,180.6万,13.9万,7.2万,6.0万,1.4万,8879,4511,否
1oK411N7Rr,8月,老坛胡说,个人认证,228.6万,197,老 坛 胡 说 之 死,音乐,原创音乐,合作视频,自制,"[原创歌曲, 搞笑, 音乐, 原创音乐, 老坛胡说, 新人]",04:59,169.6万,21.6万,11.8万,4.9万,4568,8692,1.6万,否
185411a7Rg,8月,力元君,个人认证,272.3万,330,5000人一起制作的rap什么水平？,音乐,原创音乐,普通视频,自制,"[freestyle, 嘻哈, 原创歌曲, RAP, 搞笑, 音乐, 原创音乐, 恶搞, M...",02:38,150.7万,16.8万,6.5万,3.3万,4771,4209,7227,否
1aK411T7R2,8月,音乐制作人Kurt,个人认证,146.7万,61,100万粉达成！感谢这两个月以来关注我的所有朋友！,音乐,原创音乐,活动作品,自制,"[原创歌曲, 演奏, 音乐, 原创音乐, bilibili夏日音乐季, KHS, 百万粉, ...",02:13,141.4万,21.5万,9.9万,2.8万,4887,3701,6477,否
1yT4y1L7jg,8月,石玺彤彤彤彤,个人认证,26.1万,40,【石玺彤】rapper小石的《就算你活在二次元》完整版 ，二次元的小快乐。,音乐,原创音乐,活动作品,自制,"[说唱新世代, HIPHOP, 原创歌曲, RAP, 说唱, 音乐, 原创音乐, 石玺彤]",03:17,128.0万,8.8万,5.1万,3.4万,9583,2611,1823,否
